Part 1: Scraping and Saving HTML Content


1: Navigated to https://sfbay.craigslist.org/search/zip

2:  Interact with the Page-Sorting:
URL when the page is sorted - newest: https://sfbay.craigslist.org/search/zip?sort=date#search=1~gallery~0~0
URL when the page is sorted - oldest: https://sfbay.craigslist.org/search/zip?sort=dateoldest#search=1~gallery~0~0

Changing the sorting criteria changes the URL - sort=date for newest whereas sort=dateoldest for oldest
Sorting area can be changed by modifying the sort='date' or 'dateoldest in URL in browser's address bar

Changing the sort order is a GET request. It's part of the URL and is not hidden.

And the variable associated is sort.

3: 
URL for page 1 : https://sfbay.craigslist.org/search/zip?sort=date#search=1~gallery~0~0

URL for page 2 : https://sfbay.craigslist.org/search/zip?sort=date#search=1~gallery~1~0

URL for page 3 : https://sfbay.craigslist.org/search/zip?sort=date#search=1~gallery~2~0

Only one number changes when you move pages on the URL which is the number next to 'gallery~'. The number is one less t3:han the page number. For instance, page 1 has 0 next to 'gallery~' in URL and page 2 has 1 there. This number can be changed accordingly to navigate between pages directly from URL

In [133]:
from bs4 import BeautifulSoup
import requests
import time
import re
import os

In [42]:
time.sleep(10)
# headers = {'User-agent': 'Mozilla/5.0'}

url = 'https://sfbay.craigslist.org/search/zip?sort=date#search=1~gallery~0~0'
page = requests.get(url) #headers=headers)

print(page)


<Response [200]>


In [44]:
soup = BeautifulSoup(page.content, 'html.parser')

# Look at the unmodified source.
print(soup)

<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="craigslist" property="og:site_name"/>
<meta content="preview" name="twitter:card"/>
<meta content="SF bay area free stuff - craigslist" property="og:title"/>
<meta content="SF bay area free stuff - craigslist" name="description"/>
<meta content="SF bay area free stuff - craigslist" property="og:description"/>
<meta content="https://sfbay.craigslist.org/search/zip" property="og:url"/>
<title>SF bay area free stuff - craigslist</title>
<link href="https://sfbay.craigslist.org/search/zip" rel="canonical"/>
<link href="https://sfbay.craigslist.org/search/zip" hreflang="x-default" rel="alternate"/>
<link href="/favicon.ico" id="favicon" rel="icon">
<script id="ld_searchpage_data" type="application/ld+json">
    {"description":"Free Stuff in SF Bay Area","@context":"https://schema.org","@type":"Search

There is 'ol' element with class 'cl-static-search-results' which has search results and 'a' element within has link to the item. The code for the same is as follows:

In [87]:
page_ol = soup.select_one("ol.cl-static-search-results")

In [88]:
print(page_ol)

<ol class="cl-static-search-results">
<li class="cl-static-hub-links">
<div>see also</div>
</li>
<li class="cl-static-search-result" title="1000 piece 2 sided puzzle 🧩">
<a href="https://sfbay.craigslist.org/eby/zip/d/union-city-1000-piece-sided-puzzle/7714897627.html">
<div class="title">1000 piece 2 sided puzzle 🧩</div>
<div class="details">
<div class="price">$0</div>
<div class="location">
                        fremont / union city / newark
                    </div>
</div>
</a>
</li>
<li class="cl-static-search-result" title="Free Phantom Dimmable Digital Ballasts">
<a href="https://sfbay.craigslist.org/sfc/zip/d/san-francisco-free-phantom-dimmable/7715591270.html">
<div class="title">Free Phantom Dimmable Digital Ballasts</div>
<div class="details">
<div class="price">$0</div>
<div class="location">
                        San Francisco
                    </div>
</div>
</a>
</li>
<li class="cl-static-search-result" title="Very comfortable vintage chair and ottaman">
<a href="h

In [92]:
page_ol_a = page_ol.select('a')
print(page_ol_a[0].get('href'))

https://sfbay.craigslist.org/eby/zip/d/union-city-1000-piece-sided-puzzle/7714897627.html


Alternatively, we can directly search for a however, here you start getting from third index. First two aren't item links.

In [90]:
page_a = soup.select("a")
print(page_a[3].get('href'))

https://sfbay.craigslist.org/sfc/zip/d/san-francisco-free-phantom-dimmable/7715591270.html


Let's go back to first method and continue.

In [91]:
#Check how links we got
print(len(page_ol_a))

360


Since I got more than 250 links here, I'll pull first 250 using a loop and make a list of the URLs.

In [113]:
#Store first 250 links in a list
url_list = []
for i in range(0, 250):
    url_list.append(page_ol_a[i].get('href'))

In [116]:
print(url_list)

['https://sfbay.craigslist.org/eby/zip/d/union-city-1000-piece-sided-puzzle/7714897627.html', 'https://sfbay.craigslist.org/sfc/zip/d/san-francisco-free-phantom-dimmable/7715591270.html', 'https://sfbay.craigslist.org/eby/zip/d/oakland-very-comfortable-vintage-chair/7708960154.html', 'https://sfbay.craigslist.org/eby/zip/d/oakland-comfy-sofa/7715586833.html', 'https://sfbay.craigslist.org/sby/zip/d/cupertino-recliner/7709238646.html', 'https://sfbay.craigslist.org/pen/zip/d/mountain-view-heavy-duty-mop/7708510254.html', 'https://sfbay.craigslist.org/pen/zip/d/mountain-view-six-dinning-chairs/7715586035.html', 'https://sfbay.craigslist.org/pen/zip/d/atherton-free-pool-stuff/7715585874.html', 'https://sfbay.craigslist.org/eby/zip/d/san-leandro-offset-printing/7709342694.html', 'https://sfbay.craigslist.org/eby/zip/d/pittsburg-free-gas-stove/7715584866.html', 'https://sfbay.craigslist.org/eby/zip/d/alameda-queen-sized-metal-bed-frame-no/7715584857.html', 'https://sfbay.craigslist.org/nby/

5. Save HTML Pages:


In [117]:
def saveString(html, filename="test.html"):
	try:
		file = open(filename, "w", encoding='utf-8')
		file.write(str(html))
		file.close()
	except Exception as ex:
		print('Error: ' + str(ex))



In [256]:
for url in url_list:
    time.sleep(5)
    print(url)
    page = requests.get(url) 
    match = re.search(r'/(\d+)\.html$', url)
    id = match.group(1)
    filename = f"{id}.html"
    saveString(page.content,filename)


https://sfbay.craigslist.org/eby/zip/d/union-city-1000-piece-sided-puzzle/7714897627.html
https://sfbay.craigslist.org/sfc/zip/d/san-francisco-free-phantom-dimmable/7715591270.html
https://sfbay.craigslist.org/eby/zip/d/oakland-very-comfortable-vintage-chair/7708960154.html
https://sfbay.craigslist.org/eby/zip/d/oakland-comfy-sofa/7715586833.html
https://sfbay.craigslist.org/sby/zip/d/cupertino-recliner/7709238646.html
https://sfbay.craigslist.org/pen/zip/d/mountain-view-heavy-duty-mop/7708510254.html
https://sfbay.craigslist.org/pen/zip/d/mountain-view-six-dinning-chairs/7715586035.html
https://sfbay.craigslist.org/pen/zip/d/atherton-free-pool-stuff/7715585874.html
https://sfbay.craigslist.org/eby/zip/d/san-leandro-offset-printing/7709342694.html
https://sfbay.craigslist.org/eby/zip/d/pittsburg-free-gas-stove/7715584866.html
https://sfbay.craigslist.org/eby/zip/d/alameda-queen-sized-metal-bed-frame-no/7715584857.html
https://sfbay.craigslist.org/nby/zip/d/sebastopol-couple-hanging-lam

Part 2: Parsing and Displaying Information from Saved HTML


Read Saved HTML Files:

In [277]:
# Loop through each file in the directory
directory = os.getcwd()

for filename in os.listdir(directory):

#     # Check if the file ends with .html

    if filename.endswith(".html"):
        # print(filename)

#         # Construct the full file path

        filepath = os.path.join(directory, filename)
        # print(filepath)

#         # Read file to string

        with open(filepath, 'r', encoding='utf-8') as file:

                html = file.read()

#         # ... use html string ...
                soup = BeautifulSoup(html, 'html.parser')
                # print(soup.prettify())
                #Get title
                # title_1 = soup.select("span#titletextonly")
                # for i in title_1:
                #         print(title_1[0].text)
                #Get image source
                # img_1 = soup.select("div.gallery")
                # img_2 = img_1.select("img")
                # print(img_2['src'])
                #Get description
                # desc = soup.select("#postingbody")
                # for i in desc:
                #         print(i.text)
                # #Get post id
                # post_id_1 = soup.select("p.postinginfo")
                # post_id = post_id_1[1]
                # print(post_id_1[1].text)
                # # #get post date
                # #it exists in first time.date timeago
                # posted_date = soup.select("time.date.timeago")
                # print(posted_date[0].title)
                # # #Get update info (If it exists, it exists in second time.date timeago )
                # update_info = soup.select('time.date.timeago')
                # print(posted_date[1].title)



            



FREE TEDDY BEARS & GALAXY ROSES FOR VALENTINE'S DAY TO SPREAD LOVE
Carboys x 2
Free 3 Bags of sand, CONSTRUCTION MATERIALS ****
leather sofa
Free
Free loose dirt for planting
PC monitor. 21 inches, VGA and DVI
Free bags of wood kindling
XBOX 360 HD AV Cable
Free King size bed
Free Workbench or Desk plus I'll give $50 cash for taking it away...
Baby Cribs-Stokke crib /Pottery Barn / double umbrella stroller! Free!
Humidifier - Holmes
Free glass coffee table
Nursing school CD\xe2\x80\x99s
Free Guitar Case
Twin quilt
4 used 37" tires
Free
Body glove bodyboard
Bamboo charging station
Free take out containers
box spring and bed frame - full size
Free Furniture - TV stand, Mirrors
Free gas stove
Free Concrete decorative posts
Pending   KENMORE canister vacuum cleaner
Free lamp
Build-your-own birdhouse
Vintage Ham Radio QSL cards
Upright Piano
Patio sliding glass doors
Heavy duty mop
Christmas decoration and wood table
coffee table  - Thick cut glass top and havy Metal base
Books & games
Sams

Part 3: Automating Login on The Old Reader


Made an account on https://theoldreader.com/

Using browser’s developer tools to inspect the page and focusing on the <form> tag involved in the login process - `<input>` fields within the login form are utf-8, authenticity_token, user[login], user[password] and commit

 

Looking at the network tab while logging in, it can be seen that POST request was made while logging in. This is to keep login information hidden.
Payload has following items:
utf-8 - ✓
authenticity_token: bmdF4HMov351nBfIZ8R9SbQQUxj6boJ9gGVP10+LEUA=
user[login]
user[password]
commit: Sign In

The input elements are the same as seen on html elements


In [220]:
headers = {'User-agent': 'Mozilla/5.0'}

url = 'https://theoldreader.com/users/sign_in'
page = requests.get(url, headers=headers)
# print(page)
soup = BeautifulSoup(page.content, 'html.parser')

In [224]:
input = soup.select_one('#new_user input[name=authenticity_token]')
authenticity_token = input.get('value')
# print(authenticity_token)

time.sleep(5) # 5s


# A session to carry the cookies and make post requests
session = requests.session()

res = session.post('https://theoldreader.com/users/sign_in', 
                        data = {'utf8' : '✓',
                                'authenticity_token' : authenticity_token,
                                'user[login]' : 'krittikasheokand2@gmail.com', 
                                'user[password]' : 'abcdefgh',
                                'commit' : 'Sign In'}, 
                        timeout = 20)

# Get the Cookies.
cookies = session.cookies.get_dict()
print(cookies)

{'_new_reader_session': 'BAh7CkkiD3Nlc3Npb25faWQGOgZFVEkiJTNkYmFlMjY3OGEyNzdmNjliMDAzZDFjNGI5MzZjZGI1BjsAVEkiGXdhcmRlbi51c2VyLnVzZXIua2V5BjsAVFsHWwZVOhpNb3BlZDo6QlNPTjo6T2JqZWN0SWQiEYwv0xSNNmR482pNWkkiIiQyYSQwNSQ2UlBpbmxUVHRkblByamRObDgwejMuBjsAVEkiDWxhbmd1YWdlBjsARjoHZW5JIhByZWRpcmVjdF90bwY7AEZJIgYvBjsARkkiEF9jc3JmX3Rva2VuBjsARkkiMU5yQ1JMMVRlVlVOMXJ3dk93RWlRTFV1R2NDMEUwd1Z4MWN2M3BSb2tBbmc9BjsARg%3D%3D--740e0cf98f3fbdefb663c5ccb26e6f118984eb74', 'i_know_you': 'FNU+Krittika', 'remember_user_token': 'BAhbB1sGVToaTW9wZWQ6OkJTT046Ok9iamVjdElkIhGML9MUjTZkePNqTVpJIiIkMmEkMDUkNlJQaW5sVFR0ZG5QcmpkTmw4MHozLgY6BkVU--44d3d50053683af35a71e5d848ae33110bbd48d7', 'signed_at': '1707419627'}


The session stores session information, user name, a token to remember user and signed at cookies to identify user, and session

In [240]:
time.sleep(5) # 5s

# To remain in-session.
page_n = requests.get('https://theoldreader.com', cookies=cookies) # or explicitly set cookies of the session (when not using session.xyz)
soup_n = BeautifulSoup(page_n.content, 'html.parser')
print(soup_n.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <!-- The above 3 meta tags *must* come first in the head; any other head content must come *after* these tags -->
  <link href="//s.theoldreader.com/assets/application-befb06d5a14978388154b51422cef437.css" media="all" rel="stylesheet" type="text/css"/>
  <link href="//s.theoldreader.com/assets/apple-touch-icon-57x57-86fe1176e14af4907a6fecfe5ca7e3f1.png" rel="apple-touch-icon-precomposed" sizes="57x57"/>
  <link href="//s.theoldreader.com/assets/apple-touch-icon-114x114-bae89acc41c93261dd962ea6ade08d22.png" rel="apple-touch-icon-precomposed" sizes="114x114"/>
  <link href="//s.theoldreader.com/assets/apple-touch-icon-72x72-f248503edfa3676f8d58af531aff7e88.png" rel="apple-touch-icon-precomposed" sizes="72x72"/>
  <link href="//s.theoldreader.com/assets/apple-touch-icon-144x144-510415291cae9b46a9ca4ac398

In the page, you can see
Bugsnag.user = {
      id: "8c2fd3148d366478f36a4d5a",
      name: "FNU Krittika",
      email: "krittikasheokand2@gmail.com"
    }

which is my email and name - identified by cookies which means I was able to log in successfully